# Sprint 2. Feature engineering och modeller


In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.impute import SimpleImputer
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# RMSE-helper 
try:
    from sklearn.metrics import root_mean_squared_error as _rmse
    def RMSE(y_true, y_pred): return _rmse(y_true, y_pred)
except Exception:
    def RMSE(y_true, y_pred): return np.sqrt(mean_squared_error(y_true, y_pred))

# Projektparametrar
TARGET_COL = "value"
AREA_COL   = "area"
TIME_COL   = "time_utc"  
WEATHER_COLS = ["temp_c","wind_ms","precip_mm"]

out_dir = Path("outputs/sprint2"); out_dir.mkdir(parents=True, exist_ok=True)


In [6]:
# Läs rådata
in_path = Path("C:\EC_skola\Projekt_i_DS\svk_forescast\data\processed\dataset\hourly_eda.parquet")
df = pd.read_parquet(in_path) if in_path.suffix == ".parquet" else pd.read_csv(in_path)

# Säkerställ tidskolumn -> DateTimeIndex, tz-aware (UTC) -> Europe/Stockholm
if TIME_COL in df.columns:
    df[TIME_COL] = pd.to_datetime(df[TIME_COL], utc=True, errors="coerce")
    df = df.set_index(TIME_COL).sort_index()
else:
    
    df = df.sort_index()
    if df.index.tz is None:
        df.index = df.index.tz_localize("UTC")

# Lokal tid för kalenderfunktioner 
df.index = df.index.tz_convert("Europe/Stockholm")

# Dubblettcheck
dups = df.index.duplicated(keep=False)
if dups.any() and AREA_COL in df.columns:
    # slå ihop dubbletter per area + tid (t.ex. medel)
    df = df.groupby([AREA_COL, df.index]).mean(numeric_only=True).reset_index(level=0).sort_index()
elif dups.any():
    df = df[~df.index.duplicated(keep="first")]

# Gapp-kontroll per area: reindexera till full timserie för att se luckor
def reindex_hourly(g):
    idx = pd.date_range(g.index.min(), g.index.max(), freq="h", tz=g.index.tz)
    return g.reindex(idx)

if AREA_COL in df.columns:
    parts = {a: reindex_hourly(g) for a, g in df.groupby(AREA_COL)}
    df_full = pd.concat(parts, names=[AREA_COL, df.index.name]).sort_index()

else:
    df_full = reindex_hourly(df)

# Rapportera luckor i target
def count_gaps(g):
    return g[TARGET_COL].isna().sum()

if AREA_COL in df_full.columns:
    gap_report = df_full.groupby(level=0).apply(count_gaps)
else:
    gap_report = count_gaps(df_full)

print("Luckor i target (antal NaN efter reindex, per area om tillämpligt):")
print(gap_report)


df = df_full.copy()


Luckor i target (antal NaN efter reindex, per area om tillämpligt):
area
SE1    0
SE2    0
SE3    0
SE4    0
dtype: int64


In [8]:
# Skapa kalenderfunktioner

AREA_COL = "area"             
TIME_LEVEL_NAME = "time_utc"   

# Gör om MultiIndex -> DatetimeIndex (tid) + area som kolumn
if isinstance(df.index, pd.MultiIndex):
   
    if TIME_LEVEL_NAME not in df.index.names:
        # välj första nivå som är datetime
        for i, name in enumerate(df.index.names):
            vals = df.index.get_level_values(i)
            if np.issubdtype(vals.dtype, np.datetime64):
                TIME_LEVEL_NAME = name
                break
        else:
            raise ValueError("Hittade ingen datetime-nivå i MultiIndex.")

    # säkerställ att AREA_COL finns som kolumn 
    if (AREA_COL in df.index.names) and (AREA_COL not in df.columns):
        df = df.reset_index(level=AREA_COL)

    # sätt index = tidsnivån 
    ts = df.index.get_level_values(TIME_LEVEL_NAME)
    df = df.copy()
    df.index = pd.DatetimeIndex(ts)
    df.index.name = TIME_LEVEL_NAME
else:
    # redan enkel index
    if getattr(df.index, "name", None) == AREA_COL:
        df = df.reset_index()

# säkerställ tz-aware och rätt zon
if not isinstance(df.index, pd.DatetimeIndex):
    raise ValueError("Index måste vara DatetimeIndex.")
if df.index.tz is None:
    df.index = df.index.tz_localize("UTC")
df.index = df.index.tz_convert("Europe/Stockholm")

# Bas-kalender 
df["hour"]  = df.index.hour
df["dow"]   = df.index.dayofweek  # måndag=0... söndag=6
df["month"] = df.index.month
df["date"]  = df.index.date
df["is_weekend"] = df["dow"] >= 5

# DST-flagga
def _is_dst(ts):
    try:
        off = ts.dst()
        return (off is not None) and (off.total_seconds() != 0)
    except Exception:
        return False
df["is_dst"] = pd.Index(df.index).map(_is_dst)

# Säsong
season_map = {
    12:"Winter",1:"Winter",2:"Winter",
     3:"Spring",4:"Spring",5:"Spring",
     6:"Summer",7:"Summer",8:"Summer",
     9:"Autumn",10:"Autumn",11:"Autumn"
}
df["season"] = df["month"].map(season_map)

# Helgdagar
try:
    import holidays
    years = pd.Index(df.index.year).unique().tolist()
    swe_holidays = holidays.Sweden(years=years)
    holi_set = set(swe_holidays.keys())  # datetime.date-objekt
    df["is_holiday"] = np.isin(df.index.date, list(holi_set))
except Exception:
    df["is_holiday"] = False

# Höglast-flagga 
df["is_peak"] = (~df["is_weekend"]) & (df["hour"].between(7, 20))



In [11]:

# Säkerställ att 'area' är kolumn, inte index 
if AREA_COL in getattr(df.index, "names", []):
    df = df.reset_index(level=AREA_COL)

# Säkerställ tidsindex & sortera per 
if not isinstance(df.index, pd.DatetimeIndex):
    raise ValueError("Index måste vara DatetimeIndex.")
if df.index.tz is None:
    df.index = df.index.tz_localize("UTC")
df.index = df.index.tz_convert("Europe/Stockholm")

time_key = df.index.name if df.index.name is not None else TIME_COL
df = df.sort_values([AREA_COL, time_key]) if AREA_COL in df.columns else df.sort_index()

# Hjälpfunktioner 
def add_lags_rolls(x: pd.DataFrame, ycol: str):
    x = x.copy()
    # Laggar (radbaserade; kräver jämn timserie för exakt 24/168h)
    x[f"{ycol}_lag1"]   = x[ycol].shift(1)
    x[f"{ycol}_lag24"]  = x[ycol].shift(24)
    x[f"{ycol}_lag168"] = x[ycol].shift(168)
    # Rolling 
    for w in (3, 24, 168):
        x[f"{ycol}_roll{w}"] = x[ycol].rolling(w, min_periods=w).mean()
    return x

def add_weather_lags(x: pd.DataFrame, weather_cols):
    x = x.copy()
    for c in weather_cols:
        if c in x.columns:
            x[f"{c}_lag1"]   = x[c].shift(1)
            x[f"{c}_lag24"]  = x[c].shift(24)
            x[f"{c}_roll24"] = x[c].rolling(24, min_periods=24).mean()
    return x

# Skapa features per område 
if AREA_COL in df.columns:
    parts = []
    for _, g in df.groupby(AREA_COL):
        g = add_lags_rolls(g, TARGET_COL)
        g = add_weather_lags(g, WEATHER_COLS)
        parts.append(g)
    feat_df = pd.concat(parts, axis=0).sort_values([AREA_COL, time_key])
else:
    feat_df = add_weather_lags(add_lags_rolls(df, TARGET_COL), WEATHER_COLS)

# Cirkadiana features
feat_df["hour_sin"] = np.sin(2*np.pi*feat_df["hour"]/24)
feat_df["hour_cos"] = np.cos(2*np.pi*feat_df["hour"]/24)
feat_df["dow_sin"]  = np.sin(2*np.pi*feat_df["dow"]/7)
feat_df["dow_cos"]  = np.cos(2*np.pi*feat_df["dow"]/7)

# Spara
feat_path = out_dir / "features.parquet"
feat_df.to_parquet(feat_path)
print("Sparat:", feat_path.as_posix())

#  Snabb visning 
cols_preview = [TARGET_COL, f"{TARGET_COL}_lag1", f"{TARGET_COL}_lag24", f"{TARGET_COL}_roll24"]
cols_preview = [c for c in cols_preview if c in feat_df.columns]
display(feat_df[[AREA_COL] + cols_preview].head(10) if AREA_COL in feat_df.columns else feat_df[cols_preview].head(10))



Sparat: outputs/sprint2/features.parquet


,area,value,value_lag1,value_lag24,value_roll24
time_utc,,,,,
2023-01-01 01:00:00+01:00,SE1,1277.760665,NaN,NaN,NaN
2023-01-01 02:00:00+01:00,SE1,1283.283639,1277.760665,NaN,NaN
2023-01-01 03:00:00+01:00,SE1,1284.192506,1283.283639,NaN,NaN
2023-01-01 04:00:00+01:00,SE1,1284.541668,1284.192506,NaN,NaN
2023-01-01 05:00:00+01:00,SE1,1272.022900,1284.541668,NaN,NaN
2023-01-01 06:00:00+01:00,SE1,1278.951150,1272.022900,NaN,NaN
2023-01-01 07:00:00+01:00,SE1,1296.125214,1278.951150,NaN,NaN
2023-01-01 08:00:00+01:00,SE1,1303.901561,1296.125214,NaN,NaN
2023-01-01 09:00:00+01:00,SE1,1324.876086,1303.901561,NaN,NaN


### Rensa rader som saknar kritiska features innan modell

In [15]:
# träningsmatris (Xy) 

import numpy as np
import pandas as pd
from collections import defaultdict

# Definiera core + optional 
core = [
    f"{TARGET_COL}_lag1", f"{TARGET_COL}_lag24", f"{TARGET_COL}_roll24",
    "hour","dow","month","is_weekend","is_dst","is_holiday","is_peak",
    "hour_sin","hour_cos","dow_sin","dow_cos",
]

optional = [
    f"{TARGET_COL}_lag168",
    *[c for c in WEATHER_COLS if c in feat_df.columns],
    *[f"{c}_lag1"   for c in WEATHER_COLS if f"{c}_lag1"   in feat_df.columns],
    *[f"{c}_lag24"  for c in WEATHER_COLS if f"{c}_lag24"  in feat_df.columns],
    *[f"{c}_roll24" for c in WEATHER_COLS if f"{c}_roll24" in feat_df.columns],
]

# Deduplicera med bibehållen ordning
def _dedup(seq):
    seen = set(); out = []
    for x in seq:
        if x not in seen:
            out.append(x); seen.add(x)
    return out

core     = _dedup([c for c in core     if c in feat_df.columns])
optional = _dedup([c for c in optional if c in feat_df.columns])

# Bygg Xy per område: droppa optional-kolumner som är helt NaN i JUST det området
nan_thresh = 0.99  # droppa feature om >99% NaN i området
dropped_by_area = defaultdict(list)
frames = []

if AREA_COL in feat_df.columns:
    # säker ordning för tidsserie-CV senare
    feat_sorted = feat_df.sort_values([AREA_COL, feat_df.index.name])
    for a, g in feat_sorted.groupby(AREA_COL):
        # beräkna NaN-andel per optional-feature
        if optional:
            na_share = g[optional].isna().mean().sort_values(ascending=False)
            bad_cols = [c for c, v in na_share.items() if v > nan_thresh]
        else:
            bad_cols = []

        if bad_cols:
            dropped_by_area[a].extend(bad_cols)

        use_cols = [TARGET_COL] + core + [c for c in optional if c not in bad_cols]
        use_cols = [c for c in use_cols if c in g.columns]  # safety

        gg = g[[AREA_COL] + use_cols].copy()

        # imputering för kvarvarande sporadiska NaN i optional
        opt_present = [c for c in optional if (c in gg.columns and c not in bad_cols)]
        if opt_present:
            gg[opt_present] = gg[opt_present].ffill().bfill()

        # släng rader där core saknas 
        gg = gg.dropna(subset=[TARGET_COL] + core)

        frames.append(gg)

    Xy = pd.concat(frames, axis=0).sort_values([AREA_COL, feat_df.index.name])
else:
    # Utan area: global NaN-kontroll
    feat_sorted = feat_df.sort_index()
    if optional:
        na_share = feat_sorted[optional].isna().mean().sort_values(ascending=False)
        bad_cols = [c for c, v in na_share.items() if v > nan_thresh]
    else:
        bad_cols = []
    use_cols = [TARGET_COL] + core + [c for c in optional if c not in bad_cols]
    use_cols = [c for c in use_cols if c in feat_sorted.columns]
    Xy = feat_sorted[use_cols].copy()
    opt_present = [c for c in optional if c in Xy.columns]
    if opt_present:
        Xy[opt_present] = Xy[opt_present].ffill().bfill()
    Xy = Xy.dropna(subset=[TARGET_COL] + core)

# Typfixar
for c in ["hour","dow","month"]:
    if c in Xy.columns: Xy[c] = Xy[c].astype("int16")
for c in ["is_weekend","is_dst","is_holiday","is_peak"]:
    if c in Xy.columns: Xy[c] = Xy[c].astype("bool")

#Rapport
print("Form efter per-område-hantering:", Xy.shape)
if AREA_COL in Xy.columns:
    print("Rader per område:")
    print(Xy.groupby(AREA_COL).size())

if dropped_by_area:
    print("\nFeatures droppade per område p.g.a. hög NaN-andel (>99%):")
    for a, cols in dropped_by_area.items():
        print(f"  [{a}] {sorted(set(cols))}")

# Snabb förhandsvisning
preview_cols = [TARGET_COL, f"{TARGET_COL}_lag1", f"{TARGET_COL}_lag24", f"{TARGET_COL}_roll24"]
preview_cols = [c for c in preview_cols if c in Xy.columns]
if AREA_COL in Xy.columns:
    display(Xy[[AREA_COL] + preview_cols].head(10))
else:
    display(Xy[preview_cols].head(10))

# Spara träningsmatris
xy_path = out_dir / "Xy_features.parquet"
Xy.to_parquet(xy_path)
print("Sparat:", xy_path.as_posix())


Form efter per-område-hantering: (87456, 29)
Rader per område:
area
SE1    21864
SE2    21864
SE3    21864
SE4    21864
dtype: int64

Features droppade per område p.g.a. hög NaN-andel (>99%):
  [SE2] ['precip_mm', 'precip_mm_lag1', 'precip_mm_lag24', 'precip_mm_roll24']


,area,value,value_lag1,value_lag24,value_roll24
time_utc,,,,,
2023-01-02 01:00:00+01:00,SE1,1335.110562,1349.193717,1277.760665,1366.664644
2023-01-02 02:00:00+01:00,SE1,1331.039471,1335.110562,1283.283639,1368.654470
2023-01-02 03:00:00+01:00,SE1,1318.299266,1331.039471,1284.192506,1370.075585
2023-01-02 04:00:00+01:00,SE1,1363.835249,1318.299266,1284.541668,1373.379484
2023-01-02 05:00:00+01:00,SE1,1398.356274,1363.835249,1272.022900,1378.643375
2023-01-02 06:00:00+01:00,SE1,1453.310259,1398.356274,1278.951150,1385.908338
2023-01-02 07:00:00+01:00,SE1,1478.612706,1453.310259,1296.125214,1393.511983
2023-01-02 08:00:00+01:00,SE1,1521.621782,1478.612706,1303.901561,1402.583659
2023-01-02 09:00:00+01:00,SE1,1528.440219,1521.621782,1324.876086,1411.065498


Sparat: outputs/sprint2/Xy_features.parquet


### Enkel modellpipeline + TimeSeriesSplit-CV

In [20]:
# TimeSeriesSplit-CV per område

# RMSE helper
try:
    from sklearn.metrics import root_mean_squared_error as _rmse
    def RMSE(y_true, y_pred): return _rmse(y_true, y_pred)
except Exception:
    def RMSE(y_true, y_pred): return np.sqrt(mean_squared_error(y_true, y_pred))

assert isinstance(Xy.index, pd.DatetimeIndex)
assert TARGET_COL in Xy.columns

# Modeller
models = {
    "Ridge": Ridge(alpha=1.0, random_state=42),
    "HGB": HistGradientBoostingRegressor(
        learning_rate=0.05,
        max_depth=6,
        max_leaf_nodes=31,
        min_samples_leaf=25,
        l2_regularization=0.0,
        early_stopping=False,
        max_iter=500,
        random_state=42
    ),
}

def ts_cv_eval(df_one: pd.DataFrame, name_prefix=""):
    df_one = df_one.sort_index().dropna(subset=[TARGET_COL])

    # Släng features som är HELT NaN i detta område
    feature_cols = [c for c in df_one.columns if c != TARGET_COL and not df_one[c].isna().all()]
    cat_cols = [AREA_COL] if AREA_COL in feature_cols else []
    num_cols = [c for c in feature_cols if c not in cat_cols]

    # Preprocess (imputer + scale/OHE)
    preprocess = ColumnTransformer(
        transformers=[
            ("num", Pipeline(steps=[
                ("imp", SimpleImputer(strategy="median")),
                ("sc",  StandardScaler(with_mean=False)),
            ]), num_cols),
            ("cat", Pipeline(steps=[
                ("imp", SimpleImputer(strategy="most_frequent")),
                ("ohe", OneHotEncoder(handle_unknown="ignore")),
            ]), cat_cols),
        ],
        remainder="drop"
    )

    X = df_one[feature_cols]
    y = df_one[TARGET_COL]

    tscv = TimeSeriesSplit(n_splits=5, gap=1)
    rows = []

    # Baselines inom samma foldar (lag1/lag24)
    for k, (tr, te) in enumerate(tscv.split(X), 1):
        y_te = y.iloc[te]
        # lag1/lag24 byggs från HELA df_one (ej bara X), men utvärderas på te
        y_pred_lag1  = y.shift(1).iloc[te]
        y_pred_lag24 = y.shift(24).iloc[te]

        m1 = y_te.notna() & y_pred_lag1.notna()
        m24 = y_te.notna() & y_pred_lag24.notna()

        mae_l1  = mean_absolute_error(y_te[m1],  y_pred_lag1[m1])
        rmse_l1 = RMSE(y_te[m1],  y_pred_lag1[m1])
        mae_l24  = mean_absolute_error(y_te[m24], y_pred_lag24[m24])
        rmse_l24 = RMSE(y_te[m24], y_pred_lag24[m24])

        rows.append({
            "model": f"{name_prefix}Naive(lag1)", "fold": k,
            "MAE": mae_l1, "RMSE": rmse_l1
        })
        rows.append({
            "model": f"{name_prefix}SNaive(lag24)", "fold": k,
            "MAE": mae_l24, "RMSE": rmse_l24
        })

    # ML-modeller i samma splitar
    for model_name, est in models.items():
        pipe = Pipeline([("prep", preprocess), ("est", est)])
        for k, (tr, te) in enumerate(tscv.split(X), 1):
            Xtr, Xte = X.iloc[tr], X.iloc[te]
            ytr, yte = y.iloc[tr], y.iloc[te]
            pipe.fit(Xtr, ytr)
            yhat = pipe.predict(Xte)
            mae  = mean_absolute_error(yte, yhat)
            rmse = RMSE(yte, yhat)
            print(f"{name_prefix}{model_name} Fold {k}: MAE={mae:.1f}, RMSE={rmse:.1f}")
            rows.append({
                "model": f"{name_prefix}{model_name}", "fold": k,
                "MAE": mae, "RMSE": rmse
            })

    out = pd.DataFrame(rows)
    # sammanfattning
    summ = (out.groupby("model")[["MAE","RMSE"]]
            .agg(MAE_mean=("MAE","mean"), MAE_std=("MAE","std"),
                 RMSE_mean=("RMSE","mean"), RMSE_std=("RMSE","std"))
            .reset_index())
    print(summ.sort_values("RMSE_mean"))
    return summ

# Kör per område om area-kolumn finns
results = []
if AREA_COL in Xy.columns:
    for a, g in Xy.groupby(AREA_COL):
        print(f"\n=== Område: {a} ===")
        results.append(ts_cv_eval(g, name_prefix=f"[{a}] "))
else:
    results.append(ts_cv_eval(Xy))

res_df = pd.concat(results, ignore_index=True)
display(res_df.sort_values(["model", "RMSE_mean"]))

res_path = out_dir / "model_cv_results.csv"
res_df.to_csv(res_path, index=False)
print("Sparat:", res_path.as_posix())




=== Område: SE1 ===
[SE1] Ridge Fold 1: MAE=15.4, RMSE=20.3
[SE1] Ridge Fold 2: MAE=17.2, RMSE=22.2
[SE1] Ridge Fold 3: MAE=14.0, RMSE=18.0
[SE1] Ridge Fold 4: MAE=14.9, RMSE=19.3
[SE1] Ridge Fold 5: MAE=15.7, RMSE=20.5
[SE1] HGB Fold 1: MAE=15.1, RMSE=19.8
[SE1] HGB Fold 2: MAE=26.2, RMSE=47.1
[SE1] HGB Fold 3: MAE=13.4, RMSE=18.3
[SE1] HGB Fold 4: MAE=12.3, RMSE=16.2
[SE1] HGB Fold 5: MAE=12.5, RMSE=16.5
                 model   MAE_mean   MAE_std  RMSE_mean   RMSE_std
2          [SE1] Ridge  15.451439  1.169269  20.038020   1.546494
1    [SE1] Naive(lag1)  17.346317  1.455128  22.471840   1.890341
0            [SE1] HGB  15.889453  5.856904  23.578423  13.220855
3  [SE1] SNaive(lag24)  48.588764  7.823137  63.679122  10.616121

=== Område: SE2 ===
[SE2] Ridge Fold 1: MAE=31.9, RMSE=44.3
[SE2] Ridge Fold 2: MAE=38.0, RMSE=49.8
[SE2] Ridge Fold 3: MAE=31.5, RMSE=41.3
[SE2] Ridge Fold 4: MAE=34.7, RMSE=45.7
[SE2] Ridge Fold 5: MAE=35.1, RMSE=45.4
[SE2] HGB Fold 1: MAE=40.0, RMSE=55.7


,model,MAE_mean,MAE_std,RMSE_mean,RMSE_std
0,[SE1] HGB,15.889453,5.856904,23.578423,13.220855
1,[SE1] Naive(lag1),17.346317,1.455128,22.471840,1.890341
2,[SE1] Ridge,15.451439,1.169269,20.038020,1.546494
3,[SE1] SNaive(lag24),48.588764,7.823137,63.679122,10.616121
4,[SE2] HGB,29.089162,8.431312,40.356358,14.121398
5,[SE2] Naive(lag1),37.782435,4.826026,50.321998,5.735590
6,[SE2] Ridge,34.257415,2.641052,45.309477,3.061918
7,[SE2] SNaive(lag24),101.180478,16.631912,132.647393,19.631454
8,[SE3] HGB,120.712189,77.641475,195.153303,171.364232
9,[SE3] Naive(lag1),206.619581,33.241280,283.907489,38.580318


Sparat: outputs/sprint2/model_cv_results.csv


## Utvärdering av modellerna per område med två skale-oberoende felmått:

#### sMAPE (Symmetric Mean Absolute Percentage Error): visar det genomsnittliga procentuella felet mellan faktisk och predikterad värde. Gör det möjligt att jämföra områden med olika storleksnivåer (t.ex. SE1 vs SE3).

#### WAPE (Weighted Absolute Percentage Error): liknar MAPE men väger felen efter totala efterfrågan.

In [23]:
# sMAPE & WAPE i samma CV 

import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor

def smape(y_true, y_pred, eps=1e-8):
    num = np.abs(y_pred - y_true)
    den = (np.abs(y_true) + np.abs(y_pred) + eps)
    return 200.0 * np.mean(num / den)

def wape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / (np.sum(np.abs(y_true)) + 1e-8)

# Bästa modell per area 
best_by_area = (res_df
    .assign(area=res_df["model"].str.extract(r"\[(SE\d)\]")[0],
            algo=res_df["model"].str.extract(r"\]\s(.*)$")[0])
    .sort_values(["area","RMSE_mean"])
    .groupby("area").first()[["algo"]])

def build_model(name):
    if name == "Ridge":
        return Ridge(alpha=1.0, random_state=42)
    return HistGradientBoostingRegressor(
        learning_rate=0.05, max_depth=6, max_leaf_nodes=31,
        min_samples_leaf=25, early_stopping=False, max_iter=500, random_state=42
    )

summary_rows = []
for a, g in Xy.groupby(AREA_COL):
    algo = best_by_area.loc[a, "algo"]
    est  = build_model(algo)

    # droppa features som är helt NaN i detta område
    feat_all = [c for c in g.columns if c != TARGET_COL]
    all_nan  = [c for c in feat_all if g[c].isna().all()]
    feat_use = [c for c in feat_all if c not in all_nan]

    # bygg kolumnlistor efter droppet
    cat_cols = [AREA_COL] if AREA_COL in feat_use else []
    num_cols = [c for c in feat_use if c not in cat_cols]

    preprocess = ColumnTransformer([
        ("num", Pipeline([("imp", SimpleImputer(strategy="median")),
                          ("sc",  StandardScaler(with_mean=False))]), num_cols),
        ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                          ("ohe", OneHotEncoder(handle_unknown="ignore"))]), cat_cols),
    ])

    X = g[feat_use].copy()   # använd rensade features
    y = g[TARGET_COL].copy()

    tscv = TimeSeriesSplit(n_splits=5, gap=1)
    last_tr, last_te = list(tscv.split(X))[-1]
    Xtr, Xte = X.iloc[last_tr], X.iloc[last_te]
    ytr, yte = y.iloc[last_tr], y.iloc[last_te]

    pipe = Pipeline([("prep", preprocess), ("est", est)])
    pipe.fit(Xtr, ytr)
    yhat = pipe.predict(Xte)

    s = smape(yte.values, yhat)
    w = wape(yte.values, yhat)
    summary_rows.append({"area": a, "best_model": algo,
                         "sMAPE_lastfold": s, "WAPE_lastfold": w})

    if all_nan:
        print(f"[{a}] Droppade all-NaN-kolumner:", sorted(all_nan))

pd.DataFrame(summary_rows)



[SE2] Droppade all-NaN-kolumner: ['precip_mm', 'precip_mm_lag1', 'precip_mm_lag24', 'precip_mm_roll24']


,area,best_model,sMAPE_lastfold,WAPE_lastfold
0,SE1,Ridge,1.260436,0.012594
1,SE2,HGB,1.398055,0.013704
2,SE3,HGB,0.846055,0.008449
3,SE4,HGB,1.195295,0.011908
